# Train disruptions predict duration 

The first attempt seems to lead to +- 70% accuracy in determining if the duration of a disruption will be longer or short than 1.5 hours, which is only 40% better than just guessing.

In [937]:
import matplotlib

font = {'family' : 'sans',
        'weight' : 'normal',
        'size'   : 10}

matplotlib.rc('font', **font)

In [938]:
import numpy as np
import pandas as pd

In [384]:
# Details regarding the data https://www.rijdendetreinen.nl/over/open-data
dfo = pd.read_csv('disruptions-2011-2018.csv')

In [385]:
dfo.sample(5)

,rdt_id,ns_lines,rdt_lines,rdt_lines_id,rdt_station_names,rdt_station_codes,cause_nl,cause_en,statistical_cause_nl,statistical_cause_en,cause_group,start_time,end_time,duration_minutes
13834,15090,Utrecht C.-Baarn,Baarn - Utrecht Centraal,44,NaN,NaN,dier op het spoor,an animal on the railway track,dier op het spoor,an animal on the railway track,external,2016-09-27 13:50:03,2016-09-27 14:29:01,24.0
7543,8799,Deventer-Almelo,Almelo - Deventer,88,NaN,NaN,herstelwerkzaamheden,repair works,herstelwerkzaamheden,repair works,engineering work,2014-07-24 08:03:01,2014-07-24 12:00:02,237.0
6623,7879,Schiphol,"Amersfoort - Schiphol Airport, Amsterdam Centr...","22,24,32,137,144,148,165",NaN,NaN,eerdere verstoring,an earlier disruption,eerdere verstoring,an earlier disruption,logistical,2014-03-06 09:10:01,2014-03-06 14:10:01,279.0
4696,5949,Den Haag C-Leiden Centraal,Den Haag Centraal - Leiden Centraal,169,NaN,NaN,wisselstoring,points failure,wisselstoring,points failure,infrastructure,2013-05-01 15:58:01,2013-05-01 17:36:01,58.0
6179,7432,Amersfoort-Ede-Wageningen,Amersfoort - Ede-Wageningen,47,NaN,NaN,herstelwerkzaamheden,repair works,herstelwerkzaamheden,repair works,engineering work,2013-12-21 23:17:02,2013-12-22 02:21:01,184.0


First thoughts:

Maybe we could predict the duration of an event based on: location, cause, day, etc.


In [386]:
dfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23969 entries, 0 to 23968
Data columns (total 14 columns):
rdt_id                  23969 non-null int64
ns_lines                23969 non-null object
rdt_lines               22619 non-null object
rdt_lines_id            22619 non-null object
rdt_station_names       9088 non-null object
rdt_station_codes       9088 non-null object
cause_nl                23784 non-null object
cause_en                23784 non-null object
statistical_cause_nl    23784 non-null object
statistical_cause_en    23784 non-null object
cause_group             23967 non-null object
start_time              23969 non-null object
end_time                23967 non-null object
duration_minutes        23967 non-null float64
dtypes: float64(1), int64(1), object(12)
memory usage: 2.6+ MB


Hence, we need to do some data filling

In [387]:
dfo.describe()

,rdt_id,duration_minutes
count,23969.000000,23967.000000
mean,13134.207435,155.901323
std,7102.761187,671.947011
min,1.000000,0.000000
25%,7245.000000,34.000000
50%,13240.000000,83.000000
75%,19243.000000,163.000000
max,25235.000000,64927.000000


In [388]:
dfo.describe(include='O')

,ns_lines,rdt_lines,rdt_lines_id,rdt_station_names,rdt_station_codes,cause_nl,cause_en,statistical_cause_nl,statistical_cause_en,cause_group,start_time,end_time
count,23969,22619,22619,9088,9088,23784,23784,23784,23784,23967,23969,23967
unique,2295,500,497,1139,1135,84,83,84,82,9,23935,23404
top,Rotterdam-Breda (HSL),Breda - Rotterdam Centraal (HSL),15,"Breda,Rotterdam Centraal","BD, RTD",defecte trein,broken down train,defecte trein,broken down train,infrastructure,2013-08-21 20:15:01,2011-07-02 09:52:02
freq,418,586,586,433,433,5555,5555,5603,5603,8328,2,4


Possibly add the features: day of the week and hour of the day for the start time. End time is not known when predicting.

In [389]:
print(dfo.columns.values)

['rdt_id' 'ns_lines' 'rdt_lines' 'rdt_lines_id' 'rdt_station_names'
 'rdt_station_codes' 'cause_nl' 'cause_en' 'statistical_cause_nl'
 'statistical_cause_en' 'cause_group' 'start_time' 'end_time'
 'duration_minutes']


In [729]:
# Extracting the relevant data 
df = dfo.copy(deep=True)
df = df[~df['duration_minutes'].isna()]
df = df[~df['rdt_lines'].isna()]
df = df[~df['cause_en'].isna()]

df['cause_group'] = df['cause_group'].fillna('unknown')
df['cause_group'] = df['cause_group'].fillna('unknown')

df = df.reset_index(drop=True)

y = df['duration_minutes']
y, bins = pd.cut(y, [0, 90, 1000000], labels=range(2), retbins=True)
#y, bins = pd.qcut(y, 2, labels=range(2), retbins=True)
y = y.values.astype(np.int8)

drop_cols = ['rdt_id', 
             'ns_lines', 
             'rdt_lines_id',
             'rdt_station_codes', 
             'cause_nl', 
             'statistical_cause_nl',
             'end_time', 
             'duration_minutes']

df = df.drop(columns=drop_cols)

In [730]:
print(bins)
pd.value_counts(y)

[      0      90 1000000]


0    12135
1    10452
dtype: int64

We want to engineer:
* Day of the week
* Month of the year
* Year 
* Hour of the day
* Number of affected station 
* Number of affected lines

In the future possibly expand with:
* Km of track affected 
* Estimate for the location in NL (let's say per province), or rural vs city area

In [731]:
start_time = df['start_time']
start_time = pd.to_datetime(start_time)

In [732]:
year = start_time.apply(lambda x: x.year - 2000).values.astype(np.int8)
month = start_time.apply(lambda x: x.month).values.astype(np.int8)
weekday = start_time.apply(lambda x: x.weekday()).values.astype(np.int8)
hour = start_time.apply(lambda x: x.hour).values.astype(np.int8)

In [733]:
#df['year'] = year
# df['monthSin'] = np.sin(2*np.pi*weekday/11)
# df['monthCos'] = np.cos(2*np.pi*month/11)
# df['weekdaySin'] = np.sin(2*np.pi*weekday/6)
# df['weekdayCos'] = np.cos(2*np.pi*weekday/6)
df['hourSin'] = np.sin(2*np.pi*hour/23)
df['hourCos'] = np.cos(2*np.pi*hour/23)
df = df.drop(columns=['start_time'])

In [734]:
df

,rdt_lines,rdt_station_names,cause_en,statistical_cause_en,cause_group,hourSin,hourCos
0,Amersfoort - Apeldoorn,NaN,by police orders,by police orders,external,-2.449294e-16,1.000000
1,Amersfoort - Ede-Wageningen,NaN,by police orders,by police orders,external,0.000000e+00,1.000000
2,Breda - Roosendaal,NaN,copper theft,copper theft,external,9.976688e-01,-0.068242
3,Eindhoven - Venlo,NaN,person hit by a train,person hit by a train,accidents,9.422609e-01,-0.334880
4,Leiden Centraal - Utrecht Centraal,NaN,broken down train,broken down train,rolling stock,9.422609e-01,-0.334880
...,...,...,...,...,...,...,...
22582,Haarlem - Leiden Centraal,"Heemstede-Aerdenhout,Hillegom,Haarlem,Leiden C...",broken down train,broken down train,rolling stock,6.310879e-01,-0.775711
22583,Den Haag Centraal - Leiden Centraal,"De Vink,Den Haag Centraal,Den Haag Mariahoeve,...",disruption elsewhere,disruption elsewhere,logistical,6.310879e-01,-0.775711
22584,Amsterdam Centraal - Haarlem,"Amsterdam Centraal,Amsterdam Sloterdijk,Haarle...",disruption elsewhere,disruption elsewhere,logistical,3.984011e-01,-0.917211
22585,Rotterdam Centraal - Schiphol Airport (HSL),"Rotterdam Centraal,Schiphol Airport",broken down train,broken down train,rolling stock,1.361666e-01,-0.990686


In [735]:
df = pd.concat([df, pd.get_dummies(df['cause_group'], prefix='cause')], axis=1)
df = df.drop(columns=['cause_group'])

In [736]:
df['affected_lines'] = df['rdt_lines'].apply(lambda x: len(x.split(',')))
df = df.drop(columns=['rdt_lines'])

In [737]:
a = pd.value_counts(df['statistical_cause_en']) < 50
a = dict(a)
temp = pd.get_dummies(df['statistical_cause_en'].apply(lambda x: ('other' if a[x] else x)),
                      prefix='statcause')

df = pd.concat([df, temp], axis=1)

In [738]:
df = df.drop(columns=['rdt_station_names', 'cause_en', 'statistical_cause_en'])

In [739]:
X = df

In [740]:
assert(len(X) == len(y ))

In [741]:
from sklearn.model_selection import train_test_split

In [742]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [743]:
import seaborn as sea

In [845]:
pd.DataFrame.corrwith()

<module 'pandas' from 'C:\\Users\\vince\\Anaconda3\\lib\\site-packages\\pandas\\__init__.py'>

In [844]:
plt.figure(figsize=(12,12))
pd.concat([X, pd.DataFrame(y)], axis=1).corr()

,hourSin,hourCos,cause_accidents,cause_engineering work,cause_external,cause_infrastructure,cause_logistical,cause_rolling stock,cause_staff,cause_unknown,...,statcause_signal and level crossing failure,statcause_signal failure,statcause_signalling and points failure,statcause_slippery railway tracks,statcause_stranded train,statcause_strike,statcause_tree on the track,statcause_unexpected engineering works,statcause_weather circumstances,0
hourSin,1.000000,-0.205204,-0.047292,0.098091,-0.064586,0.054823,-0.015460,-0.035866,0.054268,-0.019594,...,0.006337,0.035763,0.000584,0.047409,-0.026662,0.048353,-0.019127,0.001638,-0.002131,0.071521
hourCos,-0.205204,1.000000,0.073754,0.086407,0.033657,-0.072635,-0.024809,-0.041129,0.025956,0.005032,...,-0.002837,-0.033939,-0.016862,-0.005521,0.022546,0.032353,-0.005896,0.022564,0.001981,0.013299
cause_accidents,-0.047292,0.073754,1.000000,-0.090897,-0.120678,-0.278256,-0.079230,-0.232331,-0.026333,-0.057318,...,-0.059912,-0.126268,-0.055664,-0.027715,-0.052761,-0.018763,-0.028381,-0.031999,-0.032388,0.190703
cause_engineering work,0.098091,0.086407,-0.090897,1.000000,-0.078832,-0.181769,-0.051757,-0.151769,-0.017202,-0.037443,...,-0.039137,-0.082484,-0.036362,-0.018105,-0.034466,-0.012257,-0.018540,0.352040,-0.021157,0.038281
cause_external,-0.064586,0.033657,-0.120678,-0.078832,1.000000,-0.241324,-0.068714,-0.201494,-0.022838,-0.049711,...,-0.051960,-0.109509,-0.048276,-0.024036,-0.045758,-0.016272,0.235182,-0.027752,-0.028089,-0.148343
cause_infrastructure,0.054823,-0.072635,-0.278256,-0.181769,-0.241324,1.000000,-0.158439,-0.464599,-0.052658,-0.114621,...,0.215312,0.453785,0.200045,-0.055422,-0.105507,-0.037520,-0.056755,-0.063990,-0.064767,0.159395
cause_logistical,-0.015460,-0.024809,-0.079230,-0.051757,-0.068714,-0.158439,1.000000,-0.132290,-0.014994,-0.032637,...,-0.034114,-0.071897,-0.031695,-0.015781,-0.030042,-0.010683,-0.016160,-0.018220,-0.018442,0.046236
cause_rolling stock,-0.035866,-0.041129,-0.232331,-0.151769,-0.201494,-0.464599,-0.132290,1.000000,-0.043967,-0.095703,...,-0.100034,-0.210828,-0.092941,-0.046275,0.227093,-0.031328,-0.047388,-0.053429,-0.054077,-0.291716
cause_staff,0.054268,0.025956,-0.026333,-0.017202,-0.022838,-0.052658,-0.014994,-0.043967,1.000000,-0.010847,...,-0.011338,-0.023896,-0.010534,-0.005245,-0.009985,0.712521,-0.005371,-0.006056,-0.006129,0.060895
cause_unknown,-0.019594,0.005032,-0.057318,-0.037443,-0.049711,-0.114621,-0.032637,-0.095703,-0.010847,1.000000,...,-0.024679,-0.052013,-0.022929,-0.011417,-0.021734,-0.007729,-0.011691,-0.013181,-0.013341,0.004679


<Figure size 864x864 with 0 Axes>

In [803]:
from sklearn import tree
from sklearn import ensemble
from sklearn import neighbors
from sklearn import svm
from sklearn import svm

#tree = tree.DecisionTreeClassifier(min_samples_leaf=80),
rf = ensemble.RandomForestClassifier(min_samples_leaf=20),

#svml = svm.LinearSVC(max_iter=1000, C=.01),
#svm = svm.SVC(max_iter=2000, C=1)

models = [
    ('tree', tree),
    ('rf', rf), 
    ('knn', knn) 
  #  ('svml', svml),
  #  ('svm', svm)
]

# predictions = []
# for model in models:
#     model.fit(X_train, y_train)
#     predictions.append(model.predict(X_train))

# predictions = pd.DataFrame(predictions).T

In [833]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
clf1 = tree.DecisionTreeClassifier(min_samples_leaf=80)
clf2 = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=80)
clf3 = neighbors.KNeighborsClassifier(25)
clf4 = svm.LinearSVC(max_iter=2000, C=.01)
clf5 = svm.SVC(max_iter=2000, C=1)

In [840]:
eclf1 = VotingClassifier(estimators=[('tree', clf1),
                                     ('rf', clf2), 
                                     ('knn', clf3)], voting='soft')

eclf1 = eclf1.fit(X_train, y_train)

In [841]:
np.mean(eclf1.predict(X_train) == y_train)

0.715185057073677

In [842]:
np.mean(eclf1.predict(X_val) == y_val)

0.701715550636414

In [843]:
np.mean(eclf1.predict(X_test) == y_test)

0.7003098716246127

In [876]:
from sklearn.preprocessing import StandardScaler  # doctest: +SKIP
scaler = StandardScaler()  # doctest: +SKIP
# Don't cheat - fit only on training data
scaler.fit(X_train)  # doctest: +SKIP
X_train = scaler.transform(X_train)  # doctest: +SKIP
# apply same transformation to test data
X_val = scaler.transform(X_val)  # doctest: +SKIP
X_test = scaler.transform(X_test)  # doctest: +SKIP

In [934]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=100,
                    learning_rate='adaptive',
                    hidden_layer_sizes=(5,),
                    max_iter=5000)
clf.fit(X_train, y_train)

print(clf.score(X_train, y_train))
print(clf.score(X_val, y_val))

0.7076444136976825
0.7042058660763697


In [935]:
print(clf.score(X_test, y_test))

0.6996458610004427


In [936]:
import numpy as np